This script takes a table with several thousand trend measurements (paleocurrent directions) for an unknown river and evaluates whether the river's bimodality is statistically significant at different sample densities.

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import sys
import seaborn as sns
from scipy.stats import norm, gaussian_kde

folder_path = os.getcwd()
sys.path.append(folder_path + '/tangent and angle code')
from rose_diagram import *
from check_bimodality import *
from transform_angles import *
from closest_compass_direction import *

# Function definitions

In [ ]:
def parse_list_of_strings(input_list):
    result = []
    for item in input_list:
        # Remove brackets and split the string into numbers
        numbers = [int(num) for num in item.strip('[]').split()]
        # Append the list of numbers to the result
        result.append(numbers)
    return result

# Import data

Copy and paste your file path below. 
This is a .csv file containing points and flow directions generated by the river-processing toolbox available on GitHub (https://github.com/ksacahas/river-processing). 
If you are using a file you have created yourself, please check the "Generate sample cohorts" section below. The following lines of code may need to be modified:

coords = np.array(parse_list_of_strings(sampled_data[:, 1])) <br>
slopes = np.expand_dims(sampled_data[:, 6], axis=1) <br>
angles = np.expand_dims(sampled_data[:, 7], axis=1) <br>

If necessary, adjust "coords" to import the coordinates and adjust the column numbers for "slopes" and "angles" to the appropriate values. 

In [ ]:
file_name = 'data_file'
file_path = folder_path + '/' + file_name

# Import data
data = pd.read_csv(file_path).values

# Make directory to save subsampled data

In [ ]:
dir_name = os.path.splitext(file_name)[0]
os.makedirs(dir_name, exist_ok=True)

dir_path = folder_path + '/' + dir_name

# Generate sample cohorts

Depending on how many sample points are in your data file, you may need to adjust "n_sample_choices"—e.g. if your data file only contains 500 data points, you should delete all numbers larger than 500 from "n_sample_choices". 

In [ ]:
# Define sample sizes
n_sample_choices = np.array([2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 20, 24, 25, 30, 32,
                             32, 40, 50, 60, 80, 90, 100, 120, 150, 160, 200,
                             240, 250, 300, 320,
                             400, 500, 600, 750, 800, 1000, 2500])
# Number of replicates
n_replicates = 100

for r in range(n_replicates):
    # Create a directory for each replicate
    replicate_dir = dir_path + f"/replicate_{r+1}"
    os.makedirs(replicate_dir, exist_ok=True)
        
    for n in n_sample_choices:
        subsample_list = []
        # Generate one cohort of size n for this replicate
        # Choose n random data points
        random_indices = random.sample(range(len(data)), n)
        sampled_data = data[random_indices, :]

        # Extract coordinates, slopes, and angles
        coords = np.array(parse_list_of_strings(sampled_data[:, 1]))
        slopes = np.expand_dims(sampled_data[:, 6], axis=1)
        angles = np.expand_dims(sampled_data[:, 7], axis=1)

        d = np.hstack((coords, slopes, angles))
        subsample_list.append(d)

        # Save the cohort as a CSV inside the replicate directory
        subsample_df = pd.DataFrame(np.vstack(subsample_list))
        cohort_filename = f"{os.path.splitext(file_name)[0]}_{n}_samples.csv"

        cohort_path = os.path.join(replicate_dir, cohort_filename)

        subsample_df.to_csv(cohort_path, index=False, header=False)

print("All replicates and cohorts have been generated.")


# Apply bimodality test to all replicates

In [ ]:
results = []

# Iterate through replicate directories (1 to 100)
for replicate_index in range(1, 101):
    replicate_dir = os.path.join(folder_path, dir_name, f'replicate_{replicate_index}')  # Iterate through all replicates
    if os.path.isdir(replicate_dir):
        # Read CSV files from the replicate directory
        for file_name in os.listdir(replicate_dir):
            if file_name.endswith('.csv'):
                # Read the CSV file
                file_path = os.path.join(replicate_dir, file_name)

                # Extract sample count from file name
                sample_count = int(file_name.split('_')[-2])

                bimodal = process_file(file_path, file_name, bw=8.0, plot=False)

                # Store results in a dictionary
                results.append({
                    'name': dir_name,
                    'samples': sample_count,
                    'bimodal': bimodal,
                    'replicate': replicate_index,  # Include replicate index
                    })


# Convert results to a DataFrame for easy manipulation
results_df = pd.DataFrame(results)

# Summarize results

In [ ]:
# Group by 'name', and 'samples', then calculate the bimodal ratio
bimodal_summary = (
    results_df.groupby(['name', 'samples'])  # Group by the relevant columns
    .agg(total_replicates=('bimodal', 'size'),  # Count total replicates
         bimodal_count=('bimodal', lambda x: (x == True).sum()))  # Count how many are bimodal=True
    .reset_index()  # Reset index to get a flat DataFrame
)

bimodal_summary['bimodal_percentage'] = (bimodal_summary['bimodal_count'] / bimodal_summary['total_replicates']) * 100


# Plot heat map

In [ ]:
for name, group in bimodal_summary.groupby('name'):
    # Pivot the group to have 'samples' as columns
    pivot_table = group.pivot(index='samples', columns='name', values='bimodal_percentage')

    # Plot the heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_table, annot=True, fmt=".1f", cmap="rainbow", cbar_kws={'label': 'Bimodal Percentage (%)'},
                vmin=0, vmax=100)
    plt.title(f"Heatmap for {name}")
    plt.xlabel("Name")
    plt.ylabel("Samples")
    plt.tight_layout()
    plt.show()


# Include sample density

Type the number of meanders in the next cell:

In [ ]:
meander_number = 1

In [ ]:
# Calculate sample densities
bimodal_summary['meander'] = meander_number
bimodal_summary['sample_density'] = bimodal_summary['samples'] / meander_number


# Plot heat map

In [ ]:
# Set the style for the plots
sns.set_style("whitegrid")

# Define the minimum and maximum values for the heatmap scale
vmin, vmax = 0, 100
sample_densities_to_show = [2, 3, 4, 5, 10, 20, 30, 40, 50, 100, 200, 250]

pivot_table = bimodal_summary.pivot_table(index='sample_density', 
                                       columns='meander', 
                                       values='bimodal_percentage', 
                                       fill_value=np.nan)  # Fill missing values with 0

filtered_pivot_table = pivot_table.loc[sample_densities_to_show]

# Sort rows according to your custom sample density order
pivot_table = pivot_table.reindex(sample_densities_to_show)

plt.figure(figsize=(10, 6))  # Set the figure size

# Create a heatmap with consistent color scale
ax = sns.heatmap(
    filtered_pivot_table,
    annot=True,  # Annotate with the values
    fmt=".1f",  # Format of the annotations
    cmap='rainbow',  # Rainbow color map
    cbar_kws={'label': 'Bimodal Percentage (%)'},  # Color bar label
    vmin=vmin,  # Minimum value for the color scale
    vmax=vmax,   # Maximum value for the color scale
    )
    
# Set titles and labels
plt.ylabel('Sample Density (samples per meander)', fontsize=12)

# Show the plot
plt.show()

# Analyze statistical significance

The next cell imports a file of control data. 
You can either create your own control file using the river-processing toolbox or use the default control file provided in that toolbox. 

In [ ]:
control_name = 'control_file.csv'
control_data = pd.read_csv(control_name)

In [ ]:
# Convert the pivot table to a long-format DataFrame with 3 columns: sample_density, meander, bimodal_percentage
long_format = filtered_pivot_table.reset_index().melt(id_vars='sample_density', var_name='meander', value_name='bimodal_percentage')

In [ ]:
# Initialize an empty list to store results
results = []

for _, unknown_row in long_format.iterrows():
    sample_density = unknown_row['sample_density']
    unknown_bimodal = unknown_row['bimodal_percentage']

    # Get control values for the same sample_density and meander
    control_bimodality_values = control_data[(control_data['sample_density'] == sample_density) &
                                (control_data['meander'] == meander_number)]['bimodal_percentage'].values
    control_bimodality_rates = control_bimodality_values

    # KDE of the control distribution
    kde = gaussian_kde(control_bimodality_rates, bw_method='scott')
    p_value_kde = 1 - kde.integrate_box_1d(-np.inf, unknown_bimodal)

    xs = np.linspace(0,100,num=1000)
        
    plt.figure(figsize=(8, 5))
    plt.plot(xs, kde(xs), label="KDE for control rivers")
    plt.axvline(x=unknown_bimodal, color='r', linestyle='dashed', label=f"positive bimodality % for unknown river, p={p_value_kde:.2f}")
    plt.xlabel('Percentage of positive bimodality results')
    plt.ylabel('Density')
    plt.title(f"Sample density {sample_density} samples/meander")
    plt.xlim(0,100)
    plt.legend()
    plt.show()

# Convert results to a DataFrame
results_df = pd.DataFrame(results)